# Given the elmo embeddings pickled, generate the pickled files with them

In [1]:
%load_ext autoreload

In [10]:
%autoreload

import os, random, pandas as pd, numpy as np
from sklearn.model_selection import StratifiedKFold
import pickle
import sys
sys.path.append('..')
import relation_extraction.data.utils as utils
import itertools
import h5py
# SEED = 1
# np.random.seed(SEED)
# random.seed(SEED)
RESOURCE_PATH = "/data/medg/misc/semeval_2010"
def res(path): return os.path.join(RESOURCE_PATH, path)

In [3]:
train_data = open(res('train.txt'))
splitted_data_border1 = utils.split_data_cut_sentence(train_data, border_size=1)

In [4]:
train_data=open(res('train.txt'))
splitted_data_border20 = utils.split_data_cut_sentence(train_data, border_size=20)

In [5]:
train_data = open(res('train.txt'))
splitted_data_border50 = utils.split_data_cut_sentence(train_data, border_size=50)

In [6]:
train_data = open(res('train.txt'))
splitted_data_border_minus1 = utils.split_data_cut_sentence(train_data)

In [15]:
elmo_embeddings = utils.get_elmo_embeddings(res('elmo/train-elmo-full.hdf5'))

In [16]:
len(elmo_embeddings[0])

8000

In [19]:
def get_dataframe(splitted_data, elmo_data):
    # now need to figure out how to create the csv file
    sen = splitted_data[0]
    rel = splitted_data[1]
    e1 = splitted_data[2]
    e2 = splitted_data[3]
    elmo_data = elmo_data[0]
    data = pd.DataFrame({'sentences': sen, 'relations': rel, 'e1_pos': e1, 'e2_pos': e2, \
                         'elmo_embeddings': elmo_data}, \
                        columns=['sentences', 'relations', 'e1_pos', 'e2_pos', 'elmo_embeddings'])
    return data

In [21]:
data_border1 = get_dataframe(splitted_data_border1, elmo_embeddings)
data_border20 = get_dataframe(splitted_data_border20, elmo_embeddings)
data_border50 = get_dataframe(splitted_data_border50, elmo_embeddings)
data_borderminus1 = get_dataframe(splitted_data_border_minus1, elmo_embeddings)

In [26]:
def pickle_data(data, filename1, filename2, SEED):
    np.random.seed(SEED)
    random.seed(SEED)
    print("seed is", SEED)
    N = len(data)
    K = 10

    splitter = StratifiedKFold(n_splits=K, shuffle=True, random_state=SEED)
    fold_indices = [indices for _, indices in splitter.split(
        data[['sentences', 'e1_pos', 'e2_pos', 'elmo_embeddings']].values,
        data['relations'].values
    )]

    assert len(fold_indices) == K, "Incorrect number of folds."
    assert len(np.concatenate(fold_indices)) == N, "Folds not comprehensive."

    splits = []
    for fold in range(K):
        test_fold  = fold_indices[fold]
        dev_fold   = fold_indices[(fold + 1) % K]

        non_train_start = fold
        non_train_end   = (fold + 2) % K

        if non_train_start < non_train_end:
            train_fold = np.concatenate(fold_indices[non_train_end:] + fold_indices[:non_train_start])
        else: train_fold = np.concatenate(fold_indices[non_train_end:non_train_start])
        joined = np.concatenate((train_fold, dev_fold, test_fold))

        assert len(joined) == N, (
            "Split not comprehensive for fold {fold}:\n"
            "  len(train_fold): {train}\n"
            "  len(dev_fold): {dev}\n"
            "  len(test_fold): {test}\n"
            "  len(joined): {joined}\n"
            "  N: {N}".format(
                N=N, fold=fold, train=len(train_fold), dev=len(dev_fold), test=len(test_fold), joined=len(joined)
            )
        )

        splits.append((
            data.iloc[train_fold],
            data.iloc[dev_fold],
            data.iloc[test_fold],
        ))

    assert len(splits) == K, "Too few splits"

    #below has the dependency info with the directionalities
    #seed_1_{K}-dep-dir-fold.pkl
    with open(res('elmo/pickled-files/'+filename1.format(K=K)), mode='wb') as f: pickle.dump(splits, f)
    with open(res('elmo/pickled-files/'+filename2.format(K=K)), mode='wb') as f: pickle.dump(splits, f, protocol=2)
    #seed_1_{K}-dep-dir-fold_py2.pkl
    # below has the dependency info without the directionalities
    # with open(res('seed_1_{K}-dep-fold.pkl'.format(K=K)), mode='wb') as f: pickle.dump(splits, f)
    # with open(res('seed_1_{K}-dep-fold_py2.pkl'.format(K=K)), mode='wb') as f: pickle.dump(splits, f, protocol=2)
    # dumping old data
    # with open(res('seed_1_{K}-fold.pkl'.format(K=K)), mode='wb') as f: pickle.dump(splits, f)
    # with open(res('seed_1_{K}-fold_py2.pkl'.format(K=K)), mode='wb') as f: pickle.dump(splits, f, protocol=2)

In [28]:
pickle_data(data_border1, 'seed_5_{K}-fold-border_1.pkl', 'seed_5_{K}-fold-border_1_py2.pkl', 5)
pickle_data(data_border20, 'seed_5_{K}-fold-border_20.pkl', 'seed_5_{K}-fold-border_20_py2.pkl', 5)
pickle_data(data_border50, 'seed_5_{K}-fold-border_50.pkl', 'seed_5_{K}-fold-border_50_py2.pkl', 5)
pickle_data(data_borderminus1, 'seed_5_{K}-fold-border_-1.pkl', 'seed_5_{K}-fold-border_-1_py2.pkl', 5)

seed is 5


/scratch/conda_envs/ontology-aided-relex/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


seed is 5
seed is 5
seed is 5
